<a href="https://colab.research.google.com/github/jackiekuen2/notes-handson-ml-tf/blob/master/ch7_ExerciseQ8Q9_VotingClfStackingEnsembleMNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# a = []
# while(1):
#     a.append('1')

# Q8. Voting Classifier

In [0]:
from sklearn.datasets import fetch_openml
import numpy as np

mnist = fetch_openml('mnist_784', version=1)
mnist.target = mnist.target.astype(np.int64)

In [3]:
mnist.data.shape

(70000, 784)

In [0]:
from sklearn.model_selection import train_test_split

# Spare n=10000 for test set
X_train_val, X_test, y_train_val, y_test = train_test_split(mnist.data, mnist.target, test_size=10000, random_state=42)

# Within X_train_val, spare n=10000 for validation set
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=10000, random_state=42)

In [5]:
print(X_train.shape, X_val.shape, X_test.shape)

(50000, 784) (10000, 784) (10000, 784)


### Train various classifiers

In [0]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import LinearSVC
# from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

In [0]:
random_forest_clf = RandomForestClassifier(n_estimators=10, random_state=42)
extra_trees_clf = ExtraTreesClassifier(n_estimators=10, random_state=42)
svm_clf = LinearSVC(random_state=42)
log_clf = LogisticRegression(random_state=42)

In [8]:
estimators = [random_forest_clf, extra_trees_clf, svm_clf, log_clf]

for estimator in estimators:
    print("Training the ", estimator)
    estimator.fit(X_train, y_train)

Training the  RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)
Training the  ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features='auto',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
    

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Training the  LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=42, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


## Validation accuracy

In [9]:
[estimator.score(X_val, y_val) for estimator in estimators]

[0.9469, 0.9492, 0.8397, 0.9186]

## Individual Test set accuracy

In [10]:
[estimator.score(X_test, y_test) for estimator in estimators]

[0.9437, 0.9474, 0.8449, 0.9178]

## Combine them into Ensemble Learning

In [0]:
from sklearn.ensemble import VotingClassifier

named_estimators = [
                    ('random_forest_clf', random_forest_clf),
                    ('extra_trees_clf', extra_trees_clf),
                    ('svm_clf', svm_clf),
                    ('log_clf', log_clf)
]

### Hard Voting

In [0]:
voting_clf = VotingClassifier(named_estimators, voting='hard', n_jobs=-1)

In [13]:
# Disable svm_clf
voting_clf.set_params(svm_clf=None)
voting_clf.estimators

[('random_forest_clf',
  RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                         criterion='gini', max_depth=None, max_features='auto',
                         max_leaf_nodes=None, max_samples=None,
                         min_impurity_decrease=0.0, min_impurity_split=None,
                         min_samples_leaf=1, min_samples_split=2,
                         min_weight_fraction_leaf=0.0, n_estimators=10,
                         n_jobs=None, oob_score=False, random_state=42, verbose=0,
                         warm_start=False)),
 ('extra_trees_clf',
  ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fractio

In [0]:
# Delete svm_clf
del voting_clf.estimators[2]

In [15]:
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('random_forest_clf',
                              RandomForestClassifier(bootstrap=True,
                                                     ccp_alpha=0.0,
                                                     class_weight=None,
                                                     criterion='gini',
                                                     max_depth=None,
                                                     max_features='auto',
                                                     max_leaf_nodes=None,
                                                     max_samples=None,
                                                     min_impurity_decrease=0.0,
                                                     min_impurity_split=None,
                                                     min_samples_leaf=1,
                                                     min_samples_split=2,
                                                     min_weight_fraction_lea

In [16]:
voting_clf.voting = 'hard'
# Validation Set Accuracy
voting_clf.score(X_val, y_val)

0.9539

In [17]:
# Test set accuracy
voting_clf.score(X_test, y_test)

0.9537

Hard Voting Classifier's accuracy = 95.39%

### Soft Voting

In [18]:
voting_clf.voting = 'soft'
voting_clf.score(X_val, y_val)

0.9523

In [19]:
voting_clf.score(X_test, y_test)

0.9535

# Q9. Stacking Ensemble
- Run the individual classifiers to make predictions on the validation set
- Create a new training set with the resulting predictions: features = a set of predictions from 4 classifiers (=X_val_predictions); target is the actual class (=y_val)
- Train a classifier on the new training set

In [0]:
# Initialize an array for X_val_predictions
X_val_predictions = np.empty((len(X_val), len(estimators)), dtype=np.float32)

# Make predictions --> Create new training set using X_val
for index, estimator in enumerate(estimators):
    X_val_predictions[:, index] = estimator.predict(X_val)

In [22]:
X_val_predictions

array([[5., 5., 8., 5.],
       [8., 8., 8., 8.],
       [2., 2., 2., 2.],
       ...,
       [7., 7., 7., 7.],
       [6., 6., 6., 6.],
       [7., 7., 7., 7.]], dtype=float32)

In [23]:
# Actual image's class
y_val

array([5, 8, 2, ..., 7, 6, 7])

In [24]:
# Create a blender, e.g. Random Forest Classifier
random_forest_blender = RandomForestClassifier(n_estimators=200, oob_score=True, n_jobs=-1, random_state=42)
random_forest_blender.fit(X_val_predictions, y_val)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=-1, oob_score=True, random_state=42, verbose=0,
                       warm_start=False)

In [26]:
# Out-of-Bag Evaluation score
random_forest_blender.oob_score_

0.9501

About 95.01% training accuracy

In [0]:
# Now, repeat for test set (X_test)
X_test_predictions = np.empty((len(X_test), len(estimators)), dtype=np.float32)

# Create new training set using X_test
for index, estimator in enumerate(estimators):
    X_test_predictions[:, index] = estimator.predict(X_test)

In [0]:
y_pred_stacking = random_forest_blender.predict(X_test_predictions)

In [29]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred_stacking)

0.9502

Stacking Ensemble test set accuracy is around 95.02%, similar to the performance of individual classifiers. Voting classifer (Q8) performs better, with 95.35%+ test set accurcacy